In [ ]:
import dspy

import os

In [26]:
MODEL='openai/gpt-5-nano'
API_KEY=os.getenv('OPENAI_API_KEY')
LOCAL=True

## Learn DSPy fundamentals through conversation!

### This notebook demonstrates:

* DSPy Signatures (conversation interface)
* DSPy ReAct (tool usage & agent reasoning)
* Multi-Agent Orchestration (intelligent routing)
* DSPy History (context management)

In [39]:
# Step 1: Configure DSPy with your LLM
if not LOCAL:
    MODEL='openai/gpt-5-nano'
    LLM = dspy.LM(model=MODEL, temperature=1.0, max_tokens=16000, api_key=API_KEY)
else:
    MODEL='lm_studio/qwen/qwen3-coder-30b'
    LLM = dspy.LM(model=MODEL, api_base="http://192.168.0.49:1234/v1/", api_key="local")

dspy.configure(lm=LLM)

In [28]:
LLM("Hi there, can you send a greeting and a quick introduction of yourself!?")

["Hello there! It's nice to meet you! 🌟\n\nI'm Qwen, a large language model developed by Tongyi Lab. I can help with various tasks like answering questions, creating text, logical reasoning, programming, and more. I'm designed to be a versatile AI assistant that can communicate in multiple languages and provide useful information and support.\n\nFeel free to ask me anything you'd like to know or any questions you might have - I'm here to help!"]

In [29]:
qa = dspy.ChainOfThought('question -> answer')
response = qa(question="What Year was The Eiffel Tower built?")
print(f'{MODEL}:', response.answer)
print(response.reasoning)

lm_studio/qwen/qwen3-coder-30b: The Eiffel Tower was built between 1887 and 1889, with completion in March 1889.
The Eiffel Tower was built between 1887 and 1889. It was constructed as the entrance arch for the 1889 World's Fair (Exposition Universelle) held in Paris to celebrate the 100th anniversary of the French Revolution. The tower was designed by engineer Gustave Eiffel, and construction began in January 1887, with the structure being completed in March 1889.


In [30]:
print(f'There are {len(LLM.history)} items in the history\n')  

print(LLM.history[-1].keys(),'\n')  # access the last call to the LM, with all metadata

for item in LLM.history:
    for key, value in item.items():
        print(f'{key}: {value}')
    print('\n')


There are 2 items in the history

dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type']) 

prompt: Hi there, can you send a greeting and a quick introduction of yourself!?
messages: None
kwargs: {}
response: ModelResponse(id='chatcmpl-01gebhgi4nfh11hvjtk4l2fg', created=1760949530, model='lm_studio/qwen/qwen3-coder-30b', object='chat.completion', system_fingerprint='qwen/qwen3-coder-30b', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello there! It's nice to meet you! 🌟\n\nI'm Qwen, a large language model developed by Tongyi Lab. I can help with various tasks like answering questions, creating text, logical reasoning, programming, and more. I'm designed to be a versatile AI assistant that can communicate in multiple languages and provide useful information and support.\n\nFeel free to ask me anything you'd like to know or any questions you might have - I'm here to help!

In [ ]:
toxicity = dspy.Predict(
    dspy.Signature(
        "comment:str -> toxic: bool",
        instructions="Mark as 'toxic' if the comment includes insults, harassment, or sarcastic derogatory remarks.",
    )
)

comments = [
    "you are ugly.",
    "What a nice guy!",
    "I hate you so much",
    "Great job on that project!",
    "You're worthless",
    "Thank you for your help",
    "Stop being so stupid",
    "You did an amazing work",
    "I can't stand you",
    "I appreciate your effort"
]

for comment in comments:
    print(comment,'| Toxic?:', toxicity(comment=comment).toxic)

you are ugly. | Toxic?: True
What a nice guy! | Toxic?: False
I hate you so much | Toxic?: True
Great job on that project! | Toxic?: False
You're worthless | Toxic?: True
Thank you for your help | Toxic?: False
Stop being so stupid | Toxic?: True
You did an amazing work | Toxic?: False
I can't stand you | Toxic?: True
I appreciate your effort | Toxic?: False


In [36]:
sentences = [
    "I'm so frustrated with this terrible service and endless waiting times.",
    "The weather is disappointing, but at least we have indoor activities.",
    "The meeting was scheduled for 3 PM, which is standard for our team.",
    "I'm feeling better after getting some rest and fresh air.",
    "What an amazing day filled with wonderful surprises and joyful moments!",
    "This product is absolutely horrible and waste of money.",
    "The customer support was very helpful and solved my issue quickly.",
    "I hate this movie, it's boring and poorly acted.",
    "The restaurant had excellent food and great atmosphere.",
    "This software is confusing and difficult to use."
]

sentiment_analysis = dspy.Predict(
    dspy.Signature(
        'sentence -> sentiment: int',
        instructions= "Give me a value from 1-10 indicating the sentiment, 1 being very negative and 10 being overwhelmingly positive",
        )
    )

for sentence in sentences:
    print(sentence)
    print(f'{sentiment_analysis(sentence=sentence).sentiment} out of 10\n')

I'm so frustrated with this terrible service and endless waiting times.
2 out of 10

The weather is disappointing, but at least we have indoor activities.
4 out of 10

The meeting was scheduled for 3 PM, which is standard for our team.
7 out of 10

I'm feeling better after getting some rest and fresh air.
7 out of 10

What an amazing day filled with wonderful surprises and joyful moments!
9 out of 10

This product is absolutely horrible and waste of money.
2 out of 10

The customer support was very helpful and solved my issue quickly.
8 out of 10

I hate this movie, it's boring and poorly acted.
2 out of 10

The restaurant had excellent food and great atmosphere.
9 out of 10

This software is confusing and difficult to use.
3 out of 10



In [38]:
document = """The rain hammered against the cracked pavement as Marcus clutched his worn canvas bag, his weathered hands trembling slightly from the cold. He had been selling hot coffee and bagels on this corner for fifteen years, ever since his wife passed away, but tonight felt different—perhaps because the steady stream of commuters had thinned to just a few desperate souls seeking warmth. His small cart, tucked between the towering glass buildings, cast a faint glow through the misty darkness, and he watched as an elderly woman with a umbrella approached, her face illuminated by the yellow light. She purchased his last cup of coffee, and as she walked away, Marcus smiled, knowing that even in the rain, someone was still finding comfort in his simple offering.
The city's endless rhythm continued around him, but for this moment, he felt like he was making a difference. The rain had been falling for hours now, turning the streets into rivers of gray water that reflected the neon signs of the nearby restaurants and stores. His cart, though modest, had become a familiar presence on this corner, a small oasis in the concrete jungle where people could pause, warm themselves, and perhaps find a moment of human connection in their hurried lives.
As he wiped the condensation from his coffee cup, Marcus noticed a young man huddled under an overpass nearby, his jacket soaked through. The boy's eyes met Marcus's, and for a brief second, there was something in that look—hope, maybe desperation, or simply recognition of shared struggle. Marcus reached into his bag and pulled out another cup of coffee, offering it to the boy without hesitation. The young man accepted it with a quiet "thank you," and Marcus felt that familiar warmth spreading through his chest, the kind that only comes from knowing you've made someone's night just a little bit better.
The rain continued its relentless assault, but Marcus was no longer alone in the darkness. He had become more than just a vendor; he had become a small beacon of humanity in a city that often forgot to look up and notice the people who were trying to keep it all together, one cup of coffee at a time.
"""
print(f'original length (characters): {len(document)}')

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)
print(f'summarised length (characters): {len(response.summary)}')

print(response.summary)
# print("Reasoning:", response.reasoning)

original length (characters): 2159
summarised length (characters): 565
Marcus, a street vendor who has been selling coffee and bagels on the same corner for fifteen years since his wife's passing, experiences a meaningful moment of human connection during a rainy night. After selling his last cup to an elderly woman, he offers coffee to a young man huddled under an overpass, recognizing shared struggle and providing warmth in the cold urban environment. The story portrays Marcus as more than just a vendor—he becomes a beacon of humanity in a city that often overlooks those trying to keep it together, one cup of coffee at a time.
